买入三法核心思想是什么

- 买入条件:
    1. 当日收盘价大于前10日收盘价的均值
    2. 当日成交量大于前10日成交量的均值
    3. stochastic算法下的kline和dline, 当日kline > 当日dline, 而前一日的kline < 前一日dline;

- 卖出条件:
    - 当日收盘价小于前20收盘价均值

两种实现方式

1. 当日计算指标，如果满足条件，次日以开盘价执行相关操作；
2. 亦或者当日计算指标，但是成交量不能这么执行对吧

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import statsmodels.api as sm
import QUANTAXIS as QA
import talib as ta
import datetime #, time

In [5]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')

In [3]:
data = QA.QA_fetch_index_day_adv('000001', '1992-02-01', '2018-01-31')

In [12]:
close = data.close

In [13]:
volume = data.volume

In [9]:
ind_kdj = QA.QA_indicator_KDJ(data)

In [10]:
ind_kdj.KDJ_D

,,KDJ_K,KDJ_D,KDJ_J
date,code,,,
1992-02-17,000001,92.091049,92.091049,92.091049
1992-02-18,000001,92.091049,92.091049,92.091049
1992-02-19,000001,94.727366,92.969822,98.242455
1992-02-20,000001,96.464885,94.134843,101.124969
1992-02-21,000001,97.387829,95.219171,101.725143


In [19]:
ind = (QA.CROSS(ind_kdj.KDJ_K, ind_kdj.KDJ_D) & \
QA.CROSS(close, close.rolling(window=10).mean()) & \
QA.CROSS(volume, volume.rolling(window=10).mean()))

In [56]:
def buy_3_methods(dataframe, short=10, long=20):
    """
    1. kline上穿dline， close上穿close 10日均线，volume上穿volume10日均线, 买入信号参考;
    
    2. close下穿close 20日均线, 卖出信号参考;
    """
    kdj = QA.QA_indicator_KDJ(dataframe)
    close = dataframe.close
    volume = dataframe.volume
    buy_signal = (QA.CROSS(kdj.KDJ_K, kdj.KDJ_D) & \
                QA.CROSS(close, close.rolling(window=short).mean()) & \
                QA.CROSS(volume, volume.rolling(window=short).mean()))
    sell_signal = (QA.CROSS(close.rolling(window=long).mean(), close))
    return pd.DataFrame({'BUY':buy_signal.shift(-1).fillna(0).apply(int), 
                         'SELL':sell_signal.shift(-1).fillna(0).apply(int)})

In [18]:
init_assets = 1e7

In [57]:
ind = data.add_func(buy_3_methods)

In [59]:
ind.xs('000001', level=1).SELL.sum()

271

In [66]:
data.date[0]

Timestamp('1992-02-03 00:00:00')

In [51]:
cash = init_assets
stock_holding_volume = {}
history_table = pd.DataFrame()
count = 0
for item in data:
    daily_ind = ind.loc[item.index]
    
    if daily_ind.BUY.iloc[0] > 0:
        code = item.code[0]
        _date = item.date[0]
        buy_price = item.open.iloc[0]
        holding_volume = cash // buy_price // 100 * 100
        stock_holding_volume[code] = holding_volume
        cash -= holding_volume * buy_price
        
        # 记录交割单
        count += 1
        _order = {'datetime':_date, 'code':code, 'price':buy_price,
                 'amount':holding_volume, 'cash':cash}
        temp = pd.DataFrame(data=_order, index=[count])
        history_table = pd.concat([history_table, temp])
    
    if daily_ind.SELL.iloc[0] > 0:
        code = item.code[0]
        _date = item.date[0]
        
        if len(stock_holding_volume)>0:
            sell_price = item.open.iloc[0]
            holding_volume = stock_holding_volume[code]
            cash += holding_volume * sell_price
            stock_holding_volume.pop(code)
            
            # 记录交割单
            count += 1
            _order = {'datetime':_date, 'code':code, 'price':sell_price,
                     'amount':-1 * holding_volume, 'cash':cash}
            temp = pd.DataFrame(data=_order, index=[count])
            history_table = pd.concat([history_table, temp])

date        code  
2018-01-25  000001    3548.31
2018-01-26  000001    3558.13
2018-01-29  000001    3523.00
2018-01-30  000001    3488.01
2018-01-31  000001    3480.83
Name: close, dtype: float64

In [72]:
len({'code':'000001'}.pop('code'))

6

In [73]:
a = {'code':'000001'}
a.pop('code')
len(a)

0

In [69]:
a.

'000001'

In [70]:
a

{}